In [1]:
from __windmodel5x5DBC__ import glob_all, save_models, WindLossmodel, define_composite_wind_model
from __windmodel5x5DBC__ import define_generator, define_discriminator, define_composite_model, train
from tensorflow.keras import backend as K

import warnings
warnings.filterwarnings('ignore')
import logging
logging.getLogger().disabled = True

import pandas as pd
import numpy as np
import tensorflow.compat.v1 as tf
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "2"
tf.config.optimizer.set_jit(True)

In [2]:
resnet_n = 9 ; epochs = 500; batch = 1

home = '/media/disk3/weisting/CycleGan/Experiment5/'

dataA_norm = np.load('new_dataA_norm.npy')
dataB_norm = np.load('new_dataB_norm.npy')
print('Loaded', dataA_norm.shape, dataB_norm.shape)

EPA73_testing_PM25 = np.load(home+'348X204/EPA73_Testing_348X204_PM25.npy', mmap_mode='r')[:3000]

Train_Direc = np.concatenate([np.load('CWBEPA_Content_WIND_DIREC_graph_Training_K3.npy'),
                             np.load('CWBEPA_Content_WIND_DIREC_graph_Training_K3.npy')])

Train_Speed = np.concatenate([np.load('CWBEPA_Content_WIND_SPEED_graph_Training_K3.npy'),
                             np.load('CWBEPA_Content_WIND_SPEED_graph_Training_K3.npy')])

station_coordinate = pd.read_csv('RandomTesting/station_coordinate')
extract5_list = np.load('RandomTesting/extract5_list.npy')
extract10_list = np.load('RandomTesting/extract10_list.npy')
extract15_list = np.load('RandomTesting/extract15_list.npy')
extract20_list = np.load('RandomTesting/extract20_list.npy')

ex20_KNNhalf = np.load('ex20_KNN_half.npy', mmap_mode='r')
ex20_KNN_norm = np.load('ex20_KNN_norm.npy', mmap_mode='r')[:,:3000,:,:]

model_path = 'CWBEPA_Graph_RMSprop_Res'+str(resnet_n)+'_CycleGan_5x5DBwind/'
if not os.path.exists(model_path+'result/'):os.makedirs(model_path+'result/')

Loaded (3194, 348, 204, 1) (3194, 348, 204, 1)


In [ ]:
K.clear_session()
image_shape = dataB_norm[0].shape
g_model_AtoB = define_generator(image_shape,resnet_n)
print('generator: A -> B')
g_model_BtoA = define_generator(image_shape,resnet_n)
print('generator: B -> A')
d_model_A = define_discriminator(image_shape, d_lr=5e-5)
print('discriminator: A -> [real/fake]')
d_model_B = define_discriminator(image_shape, d_lr=5e-5)
print('discriminator: B -> [real/fake]')
c_model_AtoB = define_composite_model(g_model_AtoB, d_model_B, g_model_BtoA, image_shape, g_lr = 1e-3)
print('composite: A -> B -> [real/fake, A]')
c_model_BtoA = define_composite_model(g_model_BtoA, d_model_A, g_model_AtoB, image_shape, g_lr = 1e-3)
print('composite: B -> A -> [real/fake, B]')

WindLoss_model = WindLossmodel()
print('WindLoss_model')
c_model_wind = define_composite_wind_model(g_model_AtoB, WindLoss_model, image_shape, g_lr= 1e-5)
print('composite_wind_model')

train(d_model_A, d_model_B, g_model_AtoB, g_model_BtoA, c_model_AtoB, c_model_BtoA, c_model_wind,
      dataA_norm, dataB_norm, model_path, EPA73_testing_PM25, Train_Direc, Train_Speed,
      station_coordinate, extract5_list, extract10_list, extract15_list, extract20_list, ex20_KNN_norm, 
      ex20_KNNhalf, n_epochs=epochs, n_batch=batch, save_epochs = 1)

print('train models done')

generator: A -> B
generator: B -> A
discriminator: A -> [real/fake]
discriminator: B -> [real/fake]
composite: A -> B -> [real/fake, A]
composite: B -> A -> [real/fake, B]
WindLoss_model
composite_wind_model
3194
>00001, dALoss[1.307,5.761] dAAcc[0.341,0.387] dBLoss[1.842,1.170] dBAcc[0.095,0.700] g[25.064,23.624] wind[63.740]
>00002, dALoss[4.077,3.919] dAAcc[0.738,0.654] dBLoss[4.574,6.857] dBAcc[0.353,0.360] g[19.673,23.973] wind[100.240]
>00003, dALoss[2.412,2.047] dAAcc[0.550,0.388] dBLoss[1.416,1.765] dBAcc[0.518,0.732] g[26.205,22.549] wind[69.059]
>00004, dALoss[5.883,1.920] dAAcc[0.454,0.815] dBLoss[2.399,0.819] dBAcc[0.684,0.706] g[25.524,21.364] wind[113.111]
>00005, dALoss[1.520,0.736] dAAcc[0.483,0.364] dBLoss[0.468,0.490] dBAcc[0.651,0.580] g[19.243,18.865] wind[68.296]
>00006, dALoss[0.368,1.171] dAAcc[0.415,0.762] dBLoss[0.522,0.743] dBAcc[0.430,0.686] g[19.260,20.650] wind[65.103]
>00007, dALoss[0.590,0.869] dAAcc[0.639,0.306] dBLoss[0.577,1.406] dBAcc[0.586,0.740] g[1

>00070, dALoss[0.334,0.174] dAAcc[0.674,0.883] dBLoss[0.171,0.065] dBAcc[0.969,0.950] g[15.891,16.205] wind[100.154]
>00071, dALoss[0.212,0.159] dAAcc[0.863,0.899] dBLoss[0.213,0.112] dBAcc[0.625,0.864] g[8.846,10.842] wind[147.906]
>00072, dALoss[0.172,0.194] dAAcc[0.913,0.967] dBLoss[0.145,0.078] dBAcc[0.788,0.900] g[14.655,16.230] wind[76.437]
>00073, dALoss[0.157,0.099] dAAcc[0.829,0.921] dBLoss[0.126,0.074] dBAcc[0.946,0.966] g[14.605,14.365] wind[84.091]
>00074, dALoss[0.142,0.055] dAAcc[0.809,0.883] dBLoss[0.131,0.033] dBAcc[0.959,0.961] g[14.913,13.859] wind[81.937]
>00075, dALoss[0.088,0.071] dAAcc[0.927,0.970] dBLoss[0.189,0.056] dBAcc[0.716,0.863] g[15.326,15.836] wind[53.059]
>00076, dALoss[0.117,0.084] dAAcc[0.920,0.906] dBLoss[0.088,0.037] dBAcc[0.835,0.989] g[15.268,15.697] wind[82.256]
>00077, dALoss[0.191,0.087] dAAcc[0.865,0.940] dBLoss[0.124,0.050] dBAcc[0.856,0.977] g[14.784,14.482] wind[93.832]
>00078, dALoss[0.088,0.036] dAAcc[0.932,0.920] dBLoss[0.109,0.053] dBAc

>00141, dALoss[0.121,0.059] dAAcc[0.965,0.982] dBLoss[0.018,0.013] dBAcc[0.984,0.999] g[15.122,14.815] wind[127.970]
>00142, dALoss[0.213,0.201] dAAcc[0.776,0.887] dBLoss[0.082,0.043] dBAcc[0.945,0.999] g[14.602,14.405] wind[92.084]
>00143, dALoss[0.149,0.105] dAAcc[0.670,0.976] dBLoss[0.090,0.027] dBAcc[0.936,0.939] g[15.024,14.966] wind[67.627]
>00144, dALoss[0.066,0.014] dAAcc[0.919,0.970] dBLoss[0.146,0.062] dBAcc[0.776,0.882] g[12.470,11.899] wind[77.511]
>00145, dALoss[0.093,0.013] dAAcc[0.716,0.982] dBLoss[0.043,0.013] dBAcc[0.945,1.000] g[14.218,13.260] wind[76.757]
>00146, dALoss[0.066,0.013] dAAcc[0.900,0.969] dBLoss[0.044,0.027] dBAcc[0.965,0.974] g[12.903,12.997] wind[61.422]
>00147, dALoss[0.045,0.019] dAAcc[0.942,0.997] dBLoss[0.019,0.011] dBAcc[0.979,0.993] g[14.419,13.972] wind[60.255]
>00148, dALoss[0.047,0.027] dAAcc[0.948,0.925] dBLoss[0.017,0.037] dBAcc[0.976,0.888] g[12.417,11.897] wind[97.436]
>00149, dALoss[0.052,0.038] dAAcc[0.964,1.000] dBLoss[0.032,0.019] dBAc

>00212, dALoss[0.031,0.013] dAAcc[0.977,0.965] dBLoss[0.034,0.029] dBAcc[0.989,1.000] g[11.854,11.356] wind[108.349]
>00213, dALoss[0.053,0.018] dAAcc[0.920,0.976] dBLoss[0.020,0.044] dBAcc[0.992,0.904] g[10.824,10.378] wind[47.340]
>00214, dALoss[0.046,0.034] dAAcc[0.958,0.940] dBLoss[0.025,0.010] dBAcc[0.939,0.999] g[13.567,13.231] wind[76.237]
>00215, dALoss[0.132,0.037] dAAcc[0.814,0.941] dBLoss[0.042,0.007] dBAcc[0.998,1.000] g[10.033,9.473] wind[154.021]
>00216, dALoss[0.037,0.042] dAAcc[1.000,0.963] dBLoss[0.008,0.009] dBAcc[1.000,0.974] g[13.631,13.240] wind[94.340]
>00217, dALoss[0.028,0.016] dAAcc[0.937,0.988] dBLoss[0.031,0.006] dBAcc[0.961,0.991] g[13.307,13.074] wind[63.749]
>00218, dALoss[0.022,0.017] dAAcc[0.964,0.964] dBLoss[0.025,0.009] dBAcc[0.958,0.983] g[11.662,11.682] wind[130.583]
>00219, dALoss[0.021,0.026] dAAcc[0.970,0.998] dBLoss[0.065,0.025] dBAcc[0.901,0.993] g[9.612,9.434] wind[63.269]
>00220, dALoss[0.043,0.027] dAAcc[0.998,0.969] dBLoss[0.019,0.033] dBAcc

>00283, dALoss[0.016,0.050] dAAcc[1.000,0.937] dBLoss[0.026,0.004] dBAcc[0.945,0.995] g[10.109,9.567] wind[129.598]
>00284, dALoss[0.069,0.024] dAAcc[0.857,0.959] dBLoss[0.033,0.084] dBAcc[1.000,1.000] g[11.934,11.532] wind[132.537]
>00285, dALoss[0.144,0.085] dAAcc[0.910,1.000] dBLoss[0.035,0.013] dBAcc[1.000,1.000] g[11.416,11.723] wind[104.596]
>00286, dALoss[0.038,0.008] dAAcc[0.935,0.980] dBLoss[0.026,0.024] dBAcc[0.999,0.928] g[9.642,8.655] wind[145.720]
>00287, dALoss[0.017,0.005] dAAcc[0.999,1.000] dBLoss[0.042,0.014] dBAcc[0.913,0.942] g[10.555,10.051] wind[53.401]
>00288, dALoss[0.032,0.015] dAAcc[0.996,0.975] dBLoss[0.026,0.005] dBAcc[0.965,1.000] g[9.832,10.129] wind[60.242]
>00289, dALoss[0.023,0.014] dAAcc[0.998,1.000] dBLoss[0.012,0.015] dBAcc[1.000,1.000] g[11.928,11.597] wind[54.976]
>00290, dALoss[0.006,0.001] dAAcc[1.000,1.000] dBLoss[0.018,0.011] dBAcc[0.999,0.976] g[9.144,9.612] wind[74.172]
>00291, dALoss[0.005,0.002] dAAcc[0.998,1.000] dBLoss[0.028,0.010] dBAcc[0

...

>18669, dALoss[0.014,0.029] dAAcc[0.991,0.977] dBLoss[0.052,0.066] dBAcc[0.914,0.865] g[1.138,1.311] wind[59.033]
>18670, dALoss[0.021,0.012] dAAcc[0.973,0.990] dBLoss[0.035,0.079] dBAcc[0.949,0.796] g[1.566,2.366] wind[84.951]
>18671, dALoss[0.008,0.016] dAAcc[1.000,0.998] dBLoss[0.079,0.102] dBAcc[0.796,0.802] g[1.702,2.352] wind[154.080]
>18672, dALoss[0.019,0.013] dAAcc[1.000,1.000] dBLoss[0.357,0.047] dBAcc[0.070,0.905] g[2.308,2.405] wind[124.072]
>18673, dALoss[0.007,0.010] dAAcc[1.000,1.000] dBLoss[0.028,0.088] dBAcc[0.993,0.744] g[0.944,1.664] wind[62.064]
>18674, dALoss[0.009,0.020] dAAcc[1.000,0.997] dBLoss[0.186,0.117] dBAcc[0.430,0.543] g[1.450,1.484] wind[156.618]
>18675, dALoss[0.020,0.044] dAAcc[1.000,0.976] dBLoss[0.186,0.056] dBAcc[0.355,0.896] g[1.956,1.901] wind[84.883]
>18676, dALoss[0.013,0.011] dAAcc[1.000,0.997] dBLoss[0.019,0.032] dBAcc[0.993,0.979] g[1.541,2.314] wind[95.420]
>18677, dALoss[0.010,0.014] dAAcc[1.000,0.998] dBLoss[0.201,0.110] dBAcc[0.281,0.669]

>18741, dALoss[0.010,0.009] dAAcc[0.999,1.000] dBLoss[0.212,0.077] dBAcc[0.277,0.819] g[1.416,1.898] wind[107.578]
>18742, dALoss[0.009,0.052] dAAcc[0.990,0.913] dBLoss[0.019,0.067] dBAcc[1.000,0.858] g[1.223,1.276] wind[105.890]
>18743, dALoss[0.032,0.017] dAAcc[0.948,0.998] dBLoss[0.065,0.037] dBAcc[0.822,0.921] g[1.585,2.344] wind[93.671]
>18744, dALoss[0.015,0.009] dAAcc[1.000,1.000] dBLoss[0.117,0.056] dBAcc[0.632,0.881] g[2.039,2.552] wind[72.864]
>18745, dALoss[0.006,0.014] dAAcc[1.000,1.000] dBLoss[0.061,0.135] dBAcc[0.832,0.552] g[1.055,1.552] wind[47.729]
>18746, dALoss[0.014,0.023] dAAcc[1.000,0.997] dBLoss[0.205,0.085] dBAcc[0.232,0.784] g[1.377,2.561] wind[65.695]
>18747, dALoss[0.014,0.009] dAAcc[1.000,1.000] dBLoss[0.013,0.097] dBAcc[1.000,0.709] g[1.060,2.010] wind[147.713]
>18748, dALoss[0.005,0.026] dAAcc[1.000,0.975] dBLoss[0.063,0.045] dBAcc[0.880,0.921] g[1.610,1.333] wind[93.835]
>18749, dALoss[0.007,0.054] dAAcc[1.000,1.000] dBLoss[0.042,0.049] dBAcc[0.948,0.921]

>18813, dALoss[0.011,0.032] dAAcc[1.000,0.996] dBLoss[0.232,0.132] dBAcc[0.219,0.545] g[1.720,2.040] wind[77.683]
>18814, dALoss[0.054,0.025] dAAcc[0.938,0.995] dBLoss[0.084,0.102] dBAcc[0.727,0.649] g[1.699,3.174] wind[38.962]
>18815, dALoss[0.017,0.025] dAAcc[0.999,0.996] dBLoss[0.028,0.053] dBAcc[0.978,0.861] g[1.847,2.443] wind[73.890]
>18816, dALoss[0.021,0.015] dAAcc[0.999,0.995] dBLoss[0.049,0.111] dBAcc[0.946,0.650] g[3.297,3.381] wind[106.710]
>18817, dALoss[0.022,0.039] dAAcc[0.973,0.943] dBLoss[0.052,0.141] dBAcc[0.857,0.553] g[0.850,1.392] wind[101.445]
>18818, dALoss[0.021,0.018] dAAcc[0.976,0.997] dBLoss[0.059,0.017] dBAcc[0.887,0.987] g[1.842,1.903] wind[50.151]
>18819, dALoss[0.009,0.009] dAAcc[1.000,1.000] dBLoss[0.041,0.032] dBAcc[0.936,0.951] g[1.407,1.565] wind[44.204]
>18820, dALoss[0.006,0.012] dAAcc[1.000,1.000] dBLoss[0.021,0.059] dBAcc[0.986,0.902] g[1.073,1.313] wind[92.507]
>18821, dALoss[0.014,0.022] dAAcc[0.996,0.968] dBLoss[0.022,0.013] dBAcc[0.980,0.996] 

>18885, dALoss[0.015,0.016] dAAcc[1.000,1.000] dBLoss[0.240,0.032] dBAcc[0.172,0.954] g[2.206,3.000] wind[92.549]
>18886, dALoss[0.024,0.048] dAAcc[0.999,0.941] dBLoss[0.041,0.051] dBAcc[0.920,0.931] g[1.377,1.574] wind[91.174]
>18887, dALoss[0.037,0.019] dAAcc[0.963,0.991] dBLoss[0.142,0.092] dBAcc[0.511,0.776] g[1.616,2.149] wind[64.700]
>18888, dALoss[0.007,0.012] dAAcc[0.995,0.998] dBLoss[0.169,0.197] dBAcc[0.451,0.307] g[1.099,1.852] wind[91.312]
>18889, dALoss[0.010,0.029] dAAcc[0.989,0.956] dBLoss[0.022,0.046] dBAcc[0.990,0.955] g[1.195,1.315] wind[70.529]
>18890, dALoss[0.061,0.035] dAAcc[0.914,0.980] dBLoss[0.046,0.044] dBAcc[0.940,0.922] g[1.479,1.988] wind[101.741]
>18891, dALoss[0.012,0.011] dAAcc[0.999,1.000] dBLoss[0.043,0.091] dBAcc[0.963,0.712] g[1.568,2.189] wind[106.620]
>18892, dALoss[0.008,0.011] dAAcc[1.000,1.000] dBLoss[0.030,0.057] dBAcc[0.974,0.882] g[1.287,1.674] wind[110.715]
>18893, dALoss[0.012,0.012] dAAcc[0.999,1.000] dBLoss[0.080,0.056] dBAcc[0.791,0.907]

>18957, dALoss[0.013,0.031] dAAcc[0.997,0.940] dBLoss[0.024,0.047] dBAcc[1.000,0.959] g[1.775,1.771] wind[69.239]
>18958, dALoss[0.052,0.021] dAAcc[0.909,0.975] dBLoss[0.293,0.132] dBAcc[0.159,0.571] g[1.299,1.984] wind[77.533]
>18959, dALoss[0.010,0.016] dAAcc[0.996,0.990] dBLoss[0.016,0.089] dBAcc[0.999,0.731] g[1.046,1.642] wind[64.643]
>18960, dALoss[0.022,0.020] dAAcc[0.975,0.968] dBLoss[0.068,0.037] dBAcc[0.783,0.993] g[1.715,1.873] wind[84.376]
>18961, dALoss[0.007,0.012] dAAcc[1.000,1.000] dBLoss[0.083,0.047] dBAcc[0.737,0.961] g[1.847,1.637] wind[73.620]
>18962, dALoss[0.050,0.036] dAAcc[0.939,0.945] dBLoss[0.040,0.177] dBAcc[0.933,0.461] g[1.324,2.404] wind[59.539]
>18963, dALoss[0.014,0.009] dAAcc[0.997,1.000] dBLoss[0.043,0.035] dBAcc[0.911,0.944] g[1.440,2.179] wind[71.677]
>18964, dALoss[0.015,0.034] dAAcc[0.986,0.960] dBLoss[0.090,0.088] dBAcc[0.710,0.765] g[1.461,1.583] wind[62.612]
>18965, dALoss[0.011,0.012] dAAcc[0.998,0.993] dBLoss[0.013,0.023] dBAcc[0.998,0.999] g[